<a href="https://colab.research.google.com/github/Alsoway7/SLM/blob/main/SLM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Load the "`HuggingFaceH4/ultrachat_200k`" and "`daily_dialog`" datasets, standardize them into a unified conversation format, merge them, and tokenize the combined data using the GPT-2 tokenizer for model training.

## 集成多源数据集

### Subtask:
Load the "HuggingFaceH4/ultrachat_200k" and "daily_dialog" datasets.


**Reasoning**:
Load the "HuggingFaceH4/ultrachat_200k" (train_sft split) and "daily_dialog" datasets using `load_dataset`, and print the first entry of each to understand their structure.



In [ ]:
from datasets import load_dataset

# Load the 'train_sft' split of the ultrachat_200k dataset
ultrachat_ds = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft")

# Load the daily_dialog dataset (returns a DatasetDict)
dailydialog_ds = load_dataset("daily_dialog")

# Print the first example from both datasets to inspect structure
print("UltraChat 200k Example:")
print(ultrachat_ds[0])
print("\nDaily Dialog Example (from train split):")
print(dailydialog_ds['train'][0])

**Reasoning**:
The error `RuntimeError: Dataset scripts are no longer supported` occurs because the `daily_dialog` dataset relies on a Python loading script. To allow this, I will add `trust_remote_code=True` to the `load_dataset` function call.



In [ ]:
from datasets import load_dataset

# Load the 'train_sft' split of the ultrachat_200k dataset
ultrachat_ds = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft")

# Load the daily_dialog dataset (returns a DatasetDict)
# trust_remote_code=True is required because this dataset uses a python loading script
dailydialog_ds = load_dataset("daily_dialog", trust_remote_code=True)

# Print the first example from both datasets to inspect structure
print("UltraChat 200k Example:")
print(ultrachat_ds[0])
print("\nDaily Dialog Example (from train split):")
print(dailydialog_ds['train'][0])

**Reasoning**:
The error `RuntimeError: Dataset scripts are no longer supported` indicates that the `daily_dialog` dataset's Python loading script is blocked by the installed `datasets` library version. To bypass this, I will load the dataset from the `refs/convert/parquet` revision, which points to the auto-converted Parquet files maintained by Hugging Face, thereby avoiding the script execution.



In [ ]:
from datasets import load_dataset

# Load the 'train_sft' split of the ultrachat_200k dataset
ultrachat_ds = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft")

# Load the daily_dialog dataset using the parquet revision to avoid script execution issues
dailydialog_ds = load_dataset("daily_dialog", revision="refs/convert/parquet")

# Print the first example from both datasets to inspect structure
print("UltraChat 200k Example:")
print(ultrachat_ds[0])
print("\nDaily Dialog Example (from train split):")
print(dailydialog_ds['train'][0])

## 数据统一与合并

### Subtask:
Standardize the Daily Dialog dataset to match the UltraChat format and merge both into a single training dataset.


**Reasoning**:
Define a formatting function to transform `daily_dialog` data into the `messages` format (alternating user/assistant roles), apply it to the dataset, select the `messages` column from both datasets to ensure schema consistency, merge them using `concatenate_datasets`, and finally shuffle and inspect the result.



In [ ]:
from datasets import concatenate_datasets

# Define function to format daily_dialog to match UltraChat's messages format
def format_dailydialog(example):
    messages = []
    for i, turn in enumerate(example['dialog']):
        # Alternate roles: even indices are 'user', odd are 'assistant'
        role = 'user' if i % 2 == 0 else 'assistant'
        messages.append({'role': role, 'content': turn})
    return {'messages': messages}

# Apply the formatting to the 'train' split of daily_dialog
processed_dailydialog = dailydialog_ds['train'].map(format_dailydialog)

# Select only the 'messages' column from both datasets to ensure schema consistency for merging
ultrachat_simple = ultrachat_ds.select_columns(['messages'])
dailydialog_simple = processed_dailydialog.select_columns(['messages'])

# Merge the datasets
merged_dataset = concatenate_datasets([ultrachat_simple, dailydialog_simple])

# Shuffle the merged dataset and print info
merged_dataset = merged_dataset.shuffle(seed=42)
print(f"Merged dataset size: {len(merged_dataset)}")
print("First entry example:")
print(merged_dataset[0])

## 分词与预处理

### Subtask:
Tokenize the merged dataset using the GPT-2 tokenizer, formatting the conversation history into a single string and preparing inputs for causal language modeling.


**Reasoning**:
Load the GPT-2 tokenizer, setting the pad token to the eos token. Define a preprocessing function that formats the conversation messages into a single string (e.g., 'User: ... \nAssistant: ...') ending with the eos token, tokenizes them with padding and truncation, and creates labels for causal language modeling. Apply this function to the merged dataset and verify the output keys.



In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer for GPT-2
tokenizer = AutoTokenizer.from_pretrained("gpt2")
# GPT-2 does not have a pad token by default, so we set it to the eos_token
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    texts = []
    for messages in examples['messages']:
        conversation_text = ""
        for message in messages:
            # capitalize role (User/Assistant)
            role = message['role'].capitalize()
            content = message['content']
            conversation_text += f"{role}: {content}\n"
        # Append the eos token at the end of the conversation
        conversation_text += tokenizer.eos_token
        texts.append(conversation_text)

    # Tokenize the texts with padding and truncation
    model_inputs = tokenizer(texts, padding="max_length", truncation=True, max_length=512)

    # For Causal Language Modeling (CLM), labels are usually the same as input_ids
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

# Apply the preprocessing function to the merged dataset
tokenized_dataset = merged_dataset.map(preprocess_function, batched=True)

# Verify the processing by printing the keys of the first item
print("Keys in the processed dataset:", tokenized_dataset[0].keys())

## Model training

### Subtask:
Configure and execute the fine-tuning of the GPT-2 model on the tokenized dataset.


In [ ]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch

# Load the pre-trained GPT-2 model
model = AutoModelForCausalLM.from_pretrained("gpt2")

# Initialize the data collator
# mlm=False because we are doing Causal Language Modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Determine if GPU is available for fp16
use_fp16 = torch.cuda.is_available()

# Define training arguments
# Limiting max_steps to 500 to ensure the training completes in a reasonable time for this environment
# Removed 'overwrite_output_dir' as it caused a TypeError in previous attempts
training_args = TrainingArguments(
    output_dir="./gpt2_finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    max_steps=500,
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=use_fp16,
    logging_steps=50,
    save_steps=500,
)

# Initialize the Trainer
# Removed 'tokenizer' argument as it caused a TypeError
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# Run the training
trainer.train()

# Save the model and tokenizer explicitly since tokenizer was removed from Trainer args
trainer.save_model("./gpt2_finetuned")
tokenizer.save_pretrained("./gpt2_finetuned")

# Task
Load the initially trained model and tokenizer from the "./gpt2_finetuned" directory. Configure a second fine-tuning phase with `TrainingArguments` using a lower learning rate (e.g., 1e-5) to refine the model weights. Execute this fine-tuning using the `Trainer` and save the final model to the "./gpt2_final_finetuned" directory.

## 加载初步训练模型

### Subtask:
Load the pre-trained model and tokenizer from the `./gpt2_finetuned` directory to serve as the base for the second fine-tuning phase.


**Reasoning**:
Load the fine-tuned model and tokenizer from the local directory `./gpt2_finetuned` to prepare for the next stage of training or inference.



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the tokenizer from the saved directory
tokenizer = AutoTokenizer.from_pretrained("./gpt2_finetuned")

# Explicitly set the pad token to the eos token for GPT-2
tokenizer.pad_token = tokenizer.eos_token

# Load the fine-tuned causal language model from the saved directory
model = AutoModelForCausalLM.from_pretrained("./gpt2_finetuned")

print("Model and tokenizer loaded successfully from ./gpt2_finetuned")

## Execute model fine-tuning

### Subtask:
Configure the training arguments, initialize the Trainer, execute the fine-tuning on the dataset, and save the final model.


**Reasoning**:
Configure the training arguments with a lower learning rate and specified steps, initialize the Trainer with the model and data collator, execute the fine-tuning, and save the final model and tokenizer.



In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import torch

# Check if CUDA is available for fp16
use_fp16 = torch.cuda.is_available()

# Define training arguments for the second fine-tuning phase
training_args = TrainingArguments(
    output_dir='./gpt2_final_finetuned',
    learning_rate=1e-5,
    max_steps=50,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    fp16=use_fp16,
    logging_steps=10,
    save_steps=50,
)

# Initialize the data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# Execute the fine-tuning
trainer.train()

# Save the final fine-tuned model and tokenizer
trainer.save_model("./gpt2_final_finetuned")
tokenizer.save_pretrained("./gpt2_final_finetuned")

## Verify the final model

### Subtask:
Load the fine-tuned model from the saved directory and generate a sample response to verify functionality.


**Reasoning**:
Load the fine-tuned model and tokenizer from the './gpt2_final_finetuned' directory, move the model to the appropriate device, and generate a sample response for the input 'User: Hello, how are you?\n' to verify the model's functionality.



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the final fine-tuned tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("./gpt2_final_finetuned")
model = AutoModelForCausalLM.from_pretrained("./gpt2_final_finetuned")

# Ensure pad token is set to eos token
tokenizer.pad_token = tokenizer.eos_token

# Move model to the correct device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Define test input
input_text = "User: Hello, how are you?\n"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Generate response
outputs = model.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
)

# Decode and print the output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Response:")
print(generated_text)

## Summary:

### Data Analysis Key Findings
*   Successfully loaded the initially trained model and tokenizer from the `./gpt2_finetuned` directory, ensuring the pad token was correctly set to the EOS token.
*   Configured the second fine-tuning phase with specific hyperparameters: a learning rate of 1e-5, a batch size of 4 per device, and a limit of 50 maximum training steps.
*   Executed the training process using the `Trainer` API and successfully saved the final model artifacts to the `./gpt2_final_finetuned` directory.
*   Verified the functionality of the final model by reloading it from the saved directory and successfully generating text based on the prompt "User: Hello, how are you?".

### Insights or Next Steps
*   While the model is functionally verifying by generating text, quantitative metrics (such as perplexity or BLEU scores) should be calculated to measure the specific performance improvement gained from this second refinement phase.
*   The model is now ready for deployment or further testing; ensure that the specific prompt template used during training is consistently applied during inference to maintain output quality.


# Task
Mount Google Drive to the Colab environment. Then, create a backup directory named `my_gpt2_models` in your Drive (e.g., `/content/drive/MyDrive/my_gpt2_models`) and copy the local `./gpt2_finetuned` and `./gpt2_final_finetuned` folders into it to ensure the trained models are saved.

## 挂载 Google Drive

### Subtask:
Mount Google Drive to the Colab environment to access cloud storage.


**Reasoning**:
Import `drive` from `google.colab` and mount the drive to `/content/drive` as requested.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Reasoning**:
The previous `drive.mount` attempt failed with a `ValueError`, likely due to state inconsistency or a non-interactive environment. I will update the code to use `force_remount=True` and wrap the mounting and subsequent backup operations (creating directory and copying files) in a `try-except` block to ensure the notebook handles the failure gracefully without crashing.



In [ ]:
from google.colab import drive
import os
import shutil

try:
    # Mount Google Drive with force_remount to fix potential mount issues
    drive.mount('/content/drive', force_remount=True)

    # Define the backup directory path in Google Drive
    backup_dir = '/content/drive/MyDrive/my_gpt2_models'

    # Create the backup directory if it doesn't exist
    os.makedirs(backup_dir, exist_ok=True)
    print(f"Backup directory ensured at: {backup_dir}")

    # List of local model directories to backup
    model_dirs = ['./gpt2_finetuned', './gpt2_final_finetuned']

    for model_dir in model_dirs:
        if os.path.exists(model_dir):
            # Construct the full destination path
            dest_path = os.path.join(backup_dir, os.path.basename(model_dir))

            # Copy the directory, allowing overwrite if it exists
            shutil.copytree(model_dir, dest_path, dirs_exist_ok=True)
            print(f"Successfully backed up {model_dir} to {dest_path}")
        else:
            print(f"Warning: Source directory {model_dir} not found.")

except Exception as e:
    print(f"An error occurred during Drive mount or backup: {e}")
    print("Backup operation skipped. Please ensure you are authenticated in Colab.")

## 交互式推理

### Subtask:
Create an interactive inference loop to chat with the fine-tuned GPT-2 model.


**Reasoning**:
Implement an interactive chat loop using the fine-tuned model. I will load the model and tokenizer, set up a `while True` loop to capture user input, format it to match the training data, generate a response, and print the assistant's reply. I will include a try-except block for `EOFError` to ensure the code exits gracefully in non-interactive environments.



In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the fine-tuned model and tokenizer
model_path = "./gpt2_final_finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("Interactive Chat Mode. Type 'exit' or 'quit' to end conversation.")

while True:
    try:
        # Get user input
        user_input = input("User: ")

        # Check for exit condition
        if user_input.strip().lower() in ["exit", "quit"]:
            print("Exiting chat.")
            break

        # Format the input to match training data
        # Pattern: User: <input>\nAssistant:
        prompt = f"User: {user_input}\nAssistant:"

        # Tokenize and move to device
        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        # Generate response
        # using parameters to improve variety
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=50,
                do_sample=True,
                top_p=0.9,
                pad_token_id=tokenizer.eos_token_id
            )

        # Decode the generated tokens
        full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract the Assistant's part
        # The prompt ends with "Assistant:", so we split by that and take the last part
        if "Assistant:" in full_response:
            assistant_reply = full_response.split("Assistant:")[-1].strip()
        else:
            assistant_reply = full_response

        print(f"Assistant: {assistant_reply}\n")

    except EOFError:
        # Handle cases where input() receives no input (e.g., non-interactive execution)
        print("\nNon-interactive mode detected. Exiting loop.")
        break

# Task
Load the `test_sft` split of the "HuggingFaceH4/ultrachat_200k" dataset and apply the same preprocessing and tokenization logic used during training. Then, load the fine-tuned model from "`./gpt2_final_finetuned`" and use the `Trainer` to evaluate it on the test dataset. Calculate the Perplexity (PPL) using the formula $PPL = e^{eval\_loss}$ based on the returned evaluation loss and print the result to verify the model's performance.

## Prepare evaluation data

### Subtask:
Load the `test_sft` split of the UltraChat dataset and preprocess it using the fine-tuned tokenizer.


**Reasoning**:
Load the 'test_sft' split of the UltraChat dataset and preprocess it using the fine-tuned tokenizer from './gpt2_final_finetuned', formatting the data for evaluation.



In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load the test split of the dataset
test_dataset = load_dataset("HuggingFaceH4/ultrachat_200k", split="test_sft")

# Load the tokenizer from the final fine-tuned model directory
tokenizer = AutoTokenizer.from_pretrained("./gpt2_final_finetuned")
tokenizer.pad_token = tokenizer.eos_token

# Define the preprocessing function
def preprocess_function(examples):
    texts = []
    for messages in examples['messages']:
        conversation_text = ""
        for message in messages:
            # Format: Role: Content\n
            role = message['role'].capitalize()
            content = message['content']
            conversation_text += f"{role}: {content}\n"
        # Append eos token
        conversation_text += tokenizer.eos_token
        texts.append(conversation_text)

    # Tokenize inputs
    model_inputs = tokenizer(texts, padding="max_length", truncation=True, max_length=512)
    # Set labels for causal language modeling
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

# Apply preprocessing to the test dataset
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

# Print the size of the processed dataset
print(f"Processed test dataset size: {len(tokenized_test_dataset)}")

# Task
From the preprocessed `tokenized_test_dataset`, select a random subset of 500 samples to expedite the evaluation. Use the `Trainer` to perform an evaluation on this subset, then calculate and print the Perplexity (PPL) of the model using the formula $PPL = e^{eval\_loss}$.

## 抽取测试子集与快速计算困惑度

### Subtask:
Select a random subset of 500 samples from the preprocessed test dataset, perform evaluation to get the loss, and calculate the model's perplexity.


**Reasoning**:
Select a random subset of 500 samples from the test dataset to reduce evaluation time, then evaluate the model using the existing trainer to calculate loss and perplexity.



In [ ]:
import math

# Create a small_eval_dataset by shuffling and selecting the first 500 examples
small_eval_dataset = tokenized_test_dataset.shuffle(seed=42).select(range(2000))

# Evaluate the model on the small dataset using the existing trainer
print("Starting evaluation on small subset...")
eval_results = trainer.evaluate(eval_dataset=small_eval_dataset)

# Extract evaluation loss
eval_loss = eval_results["eval_loss"]

# Calculate Perplexity
perplexity = math.exp(eval_loss)

# Print results
print(f"Evaluation Loss: {eval_loss:.4f}")
print(f"Perplexity: {perplexity:.4f}")